This example creates a CAD geometry, converts it to a faceted neutronics geometry and obtains the trtium breeding ration (TBR)

Imports all the packages needed for the model creation and simulation

In [ ]:
import os
# more details on this package avaialbe from https://github.com/ukaea/paramak
import paramak
# more details on this package https://github.com/ukaea/neutronics_material_maker/
from neutronics_material_maker import Material
# more details on this neutronics code https://github.com/openmc-dev/openmc
import openmc

Creates a simple ring shaped geoemtry

In [ ]:

width=500

# creates a parametric shape
pf_coil = paramak.RotateStraightShape(
    points=[
        (width, width),
        (550, width),
        (550, 550),
        (500, 550)
    ],
    stp_filename = 'pf_coil.stp',
    material_tag = 'pf_coil_material'
)

pf_coil.solid

Creates a simple blanket geometry

In [ ]:
# creates another parametric shape
blanket = paramak.RotateMixedShape(
    points=[
        (538, 305, "straight"),
        (538, -305, "straight"),
        (322, -305, "spline"),
        (470, 0, "spline"),
        (322, 305, "straight")
    ],
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
    stp_filename = 'blanket.stp',
    material_tag = 'blanket_material'
)
blanket.solid

Creates a reactor from the two geometries previously made

In [ ]:
# creates a reactor object from the two components
my_reactor = paramak.Reactor([blanket, pf_coil])

my_reactor.solid

exports neutronics description and stp files

my_reactor.export_neutronics_description()
my_reactor.export_stp()

Runs Trelis which converts the Stp files to a faceted neutronics geometry (DAGMC geoemtry) with linkage between materials and volumes

In [ ]:
os.system('trelis -batch -nographics make_faceteted_neutronics_model.py')

os.system('make_watertight dagmc_notwatertight.h5m -o dagmc.h5m')

Creates an empty geometry that is later filled by the DAGMC geoemtry

In [ ]:
universe = openmc.Universe()
geom = openmc.Geometry(universe)

Creates some materials to use in the simulation, this uses the neutronics_material_maker package that can be installed with pip install neutronics_material_maker

In [ ]:
mat1 = Material(material_name='Li4SiO4',
                material_tag='blanket_material').openmc_material

mat2 = Material(material_name='copper',
                material_tag='pf_coil_material').openmc_material

mats = openmc.Materials([mat1, mat2])

Sets the number of particles to simulate and the .dagmc part loads the neutronics geometry into the empty universe previously created

In [ ]:
settings = openmc.Settings()
settings.batches = 10
settings.inactive = 0
settings.particles = 100
settings.run_mode = 'fixed source'
settings.dagmc = True 

Makes a simple point neutron source with 14MeV

In [ ]:
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
settings.source = source

Tells OpenMC to count (tally) neutrons that create tritium atoms

In [ ]:
tallies = openmc.Tallies()
tbr_tally = openmc.Tally(name='TBR')
tbr_tally.scores = ['(n,Xt)'] # where X is a wild card
tallies.append(tbr_tally)

Combines the geometry, materials, settings and tallies to make the openmc model and runs the simulation

In [ ]:
model = openmc.model.Model(geom, mats, settings, tallies)

output_filename = model.run()

Loads up the output file generated and accesses the TBR value obtained during simulation

In [ ]:
# open the results file
sp = openmc.StatePoint(output_filename)

# access the tally
tbr_tally = sp.get_tally(name='TBR')
df = tbr_tally.get_pandas_dataframe()
tbr_tally_result = df['mean'].sum()

# print result
print('The tritium breeding ratio was found, TBR = ', tbr_tally_result)